In [1]:
import os.path
import datetime
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk
import cv2
from PIL import Image, ImageTk
import face_recognition
import numpy as np
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, classification_report
import pandas as pd
import util
from test import test


class App:
    def __init__(self):
        self.main_window = tk.Tk()
        self.main_window.geometry("1200x520+350+100")
        self.main_window.title("VERIFICATION")
        self.main_window.configure(background="BLACK")

        self.login_button_main_window = util.get_button(self.main_window, 'LOGIN', 'GREEN', self.login)
        self.login_button_main_window.place(x=750, y=80)

        self.logout_button_main_window = util.get_button(self.main_window, 'LOGOUT', 'GREEN', self.logout)
        self.logout_button_main_window.place(x=750, y=180)

        self.register_new_user_button_main_window = util.get_button(self.main_window, 'REGISTER NEW USER', 'GREEN',
                                                                    self.register_new_user, fg='white')
        self.register_new_user_button_main_window.place(x=750, y=280)

        self.webcam_label = util.get_img_label(self.main_window)
        self.webcam_label.place(x=10, y=0, width=700, height=500)

        self.add_webcam(self.webcam_label)

        self.db_dir = './db'
        if not os.path.exists(self.db_dir):
            os.mkdir(self.db_dir)

        self.log_path = './log.txt'

    def add_webcam(self, label):
        if 'cap' not in self.__dict__:
            self.cap = cv2.VideoCapture(0)

        self._label = label
        self.process_webcam()

    def process_webcam(self):
        ret, frame = self.cap.read()

        if frame is not None:  # Add a check for empty frame
            self.most_recent_capture_arr = frame
            img_ = cv2.cvtColor(self.most_recent_capture_arr, cv2.COLOR_BGR2RGB)
            self.most_recent_capture_pil = Image.fromarray(img_)
            imgtk = ImageTk.PhotoImage(image=self.most_recent_capture_pil)
            self._label.imgtk = imgtk
            self._label.configure(image=imgtk)

        self._label.after(20, self.process_webcam)

    def login(self):
        label = test(
            image=self.most_recent_capture_arr,
            model_dir='C:/Users/pc/FACE RECOGNITION SYSTEM WITH LIVENESS COMPLETE/resources/anti_spoof_models',
            device_id=0
        )

        if label == 1:
            name = util.recognize(self.most_recent_capture_arr, self.db_dir)

            if name in ['unknown_person', 'no_persons_found']:
                util.msg_box('Ups...', 'Unknown user. Please register new user or try again.')
            else:
                util.msg_box('Welcome back!', 'Welcome, {}.'.format(name))
                with open(self.log_path, 'a') as f:
                    f.write('{},{},in\n'.format(name, datetime.datetime.now()))
        else:
            util.msg_box('Hey, you are a spoofer!', 'You are fake!')

        # Calculate metrics
        true_label = 1 if name not in ['unknown_person', 'no_persons_found'] else 0
        predicted_label = label
        self.calculate_metrics(true_label, predicted_label)

    def logout(self):
        label = test(
            image=self.most_recent_capture_arr,
            model_dir='C:/Users/pc/FACE RECOGNITION SYSTEM WITH LIVENESS COMPLETE/resources/anti_spoof_models',
            device_id=0
        )

        if label == 1:
            name = util.recognize(self.most_recent_capture_arr, self.db_dir)

            if name in ['unknown_person', 'no_persons_found']:
                util.msg_box('Ups...', 'Unknown user. Please register new user or try again.')
            else:
                util.msg_box('Hasta la vista!', 'Goodbye, {}.'.format(name))
                with open(self.log_path, 'a') as f:
                    f.write('{},{},out\n'.format(name, datetime.datetime.now()))
        else:
            util.msg_box('Hey, you are a spoofer!', 'You are fake!')

        # Calculate metrics
        true_label = 1 if name not in ['unknown_person', 'no_persons_found'] else 0
        predicted_label = label
        self.calculate_metrics(true_label, predicted_label)

    def register_new_user(self):
        self.register_new_user_window = tk.Toplevel(self.main_window)
        self.register_new_user_window.geometry("1200x520+370+120")
        self.register_new_user_window.title("VERIFICATION")
        self.register_new_user_window.configure(background="BLACK")
        self.accept_button_register_new_user_window = util.get_button(
            self.register_new_user_window, 'Accept', 'green', self.accept_register_new_user)
        self.accept_button_register_new_user_window.place(x=750, y=300)

        self.try_again_button_register_new_user_window = util.get_button(
            self.register_new_user_window, 'Try again', 'red', self.try_again_register_new_user)
        self.try_again_button_register_new_user_window.place(x=750, y=400)

        self.capture_label = util.get_img_label(self.register_new_user_window)
        self.capture_label.place(x=10, y=0, width=700, height=500)

        self.add_img_to_label(self.capture_label)

        self.entry_text_register_new_user = util.get_entry_text(self.register_new_user_window)
        self.entry_text_register_new_user.place(x=750, y=150)

        self.text_label_register_new_user = util.get_text_label(self.register_new_user_window, 'Please,\ninput username:')
        self.text_label_register_new_user.place(x=750, y=70)

    def try_again_register_new_user(self):
        self.register_new_user_window.destroy()

    def add_img_to_label(self, label):
        imgtk = ImageTk.PhotoImage(image=self.most_recent_capture_pil)
        label.imgtk = imgtk
        label.configure(image=imgtk)

        self.register_new_user_capture = self.most_recent_capture_arr.copy()

    def start(self):
        self.main_window.mainloop()

    def accept_register_new_user(self):
        name = self.entry_text_register_new_user.get(1.0, "end-1c")

        embeddings = face_recognition.face_encodings(self.register_new_user_capture)[0]

        file = open(os.path.join(self.db_dir, '{}.pickle'.format(name)), 'wb')
        pickle.dump(embeddings, file)

        util.msg_box('Success!', 'User was registered successfully!')

        self.register_new_user_window.destroy()

    def calculate_metrics(self, true_label, predicted_label):
        y_true = np.array([true_label])
        y_pred = np.array([predicted_label])

        cm = confusion_matrix(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        accuracy = accuracy_score(y_true, y_pred)
        classification_rep = classification_report(y_true, y_pred)

        print("Confusion Matrix:")
        print(cm)
        print("F1 Score:", f1)
        print("Precision:", precision)
        print("Recall:", recall)
        print("Accuracy:", accuracy)
        print("Classification Report:")
        print(classification_rep)
        
        # Plot confusion matrix
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="viridis", cbar=False)
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.title("Confusion Matrix")
        plt.show()

        # Plot classification report
        report_data = classification_report(y_true, y_pred, output_dict=True)
        report_df = pd.DataFrame(report_data).transpose()
        plt.figure(figsize=(6, 4))
        sns.heatmap(report_df.iloc[:-1, :].T, annot=True, cmap="viridis")
        plt.xlabel("Metrics")
        
        plt.ylabel("Classes")
        plt.title("Classification Report")
        plt.show()


if __name__ == "__main__":
    app = App()
    app.start()

ModuleNotFoundError: No module named 'face_recognition'

In [2]:
pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
     ---------------------------------------- 11.8/11.8 MB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build dlib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [73 lines of output]
  running bdist_wheel
  running build
  running build_ext
  Building extension for Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
  Invoking CMake setup: 'cmake C:\Users\ABC\AppData\Local\Temp\pip-install-pvjp1n4w\dlib_ed7bd0bb39c0471686b5a85267ebff5f\tools\python -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=C:\Users\ABC\AppData\Local\Temp\pip-install-pvjp1n4w\dlib_ed7bd0bb39c0471686b5a85267ebff5f\build\lib.win-amd64-cpython-38 -DPYTHON_EXECUTABLE=d:\ProgramData\Anaconda3\python.exe -DCMAKE_LIBRARY_OUTPUT_DIRECTORY_RELEASE=C:\Users\ABC\AppData\Local\Temp\pip-install-pvjp1n4w\dlib_ed7bd0bb39c0471686b5a85267ebff5f\build\lib.win-amd64-cpython-38 -A x64'
  -- Building for: NMake Makefiles
  CMake Error at CMakeLists.txt:5 (message):
  
  
  
    !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  
  
    You must use Visual Studio to build a python extension on win